In [13]:
"""
Kalshi API Utility Functions
=============================
This module provides signed and authenticated access to the Kalshi Trading API, including tools for working with quotes, RFQs, and account information.
It supports all major HTTP methods (GET, POST, PUT, DELETE) with RSA-based authentication using your Kalshi credentials.

Environment variables required (set in `.env`):
- PROD_KEYID / DEMO_KEYID: Your Kalshi key ID
- PROD_KEYFILE / DEMO_KEYFILE: Path to your RSA private key file
- ENV: Either "PROD" or "DEMO" to toggle environments
"""

import os
import requests
from dotenv import load_dotenv
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa

# Load environment variables from .env
load_dotenv()

ENVIRONMENT = os.getenv("ENV", "PROD")
KEY_ID = os.getenv(f"{ENVIRONMENT}_KEYID")
KEY_FILE = os.getenv(f"{ENVIRONMENT}_KEYFILE")
BASE_URL = "https://api.elections.kalshi.com/trade-api/v2"

# Load private key
with open(KEY_FILE, "rb") as key_file:
    PRIVATE_KEY = serialization.load_pem_private_key(
        key_file.read(),
        password=None,
        backend=default_backend()
    )

# Helper to generate headers for authentication
# Uses RSA-PSS to sign the message composed of timestamp, method, and path
def sign_request(method: str, path: str):
    timestamp = str(int(__import__('time').time() * 1000))
    msg = timestamp + method.upper() + path
    signature = base64.b64encode(
        PRIVATE_KEY.sign(
            msg.encode('utf-8'),
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.DIGEST_LENGTH
            ),
            hashes.SHA256()
        )
    ).decode('utf-8')

    headers = {
        'KALSHI-ACCESS-KEY': KEY_ID,
        'KALSHI-ACCESS-TIMESTAMP': timestamp,
        'KALSHI-ACCESS-SIGNATURE': signature,
        'accept': 'application/json',
        'content-type': 'application/json'
    }
    return headers

# Generic HTTP methods

# Generic GET request
# `endpoint` should be like '/communications/id'
# `params` is a dictionary of URL parameters
def kalshi_get(endpoint, params=None):
    url = BASE_URL + endpoint
    headers = sign_request("GET", endpoint)
    response = requests.get(url, headers=headers, params=params)
    response.raise_for_status()
    return response.json()

# Generic POST request
# `body` is a JSON-serializable dictionary
def kalshi_post(endpoint, body=None):
    url = BASE_URL + endpoint
    headers = sign_request("POST", endpoint)
    response = requests.post(url, headers=headers, json=body)
    response.raise_for_status()
    return response.json() if response.content else {}

# Generic PUT request
# Used for actions like confirming or accepting quotes
def kalshi_put(endpoint, body=None):
    url = BASE_URL + endpoint
    headers = sign_request("PUT", endpoint)
    response = requests.put(url, headers=headers, json=body)
    response.raise_for_status()
    return response.json() if response.content else {}

# Generic DELETE request
# Deletes the resource at the endpoint (quote, RFQ, etc.)
def kalshi_delete(endpoint):
    url = BASE_URL + endpoint
    headers = sign_request("DELETE", endpoint)
    response = requests.delete(url, headers=headers)
    response.raise_for_status()
    return response.status_code

# ========== Communications API Wrappers ==========

def get_api_version():
    """Returns the version of the Kalshi API currently in use."""
    return kalshi_get("/api_version")

def get_communications_id():
    """Returns the communications ID of the logged-in user."""
    return kalshi_get("/communications/id")

def get_quotes():
    """Returns all quotes visible to the user."""
    return kalshi_get("/communications/quotes")

def create_quote(rfq_id, yes_bid=None, no_bid=None, rest_remainder=False):
    """Creates a quote for an existing RFQ.
    Args:
        rfq_id (str): The RFQ ID you're responding to.
        yes_bid (int): Price in cents you're willing to pay for 'yes'.
        no_bid (int): Price in cents you're willing to pay for 'no'.
        rest_remainder (bool): Whether to leave rest of quantity available.
    """
    body = {
        "rfq_id": rfq_id,
        "yes_bid": yes_bid,
        "no_bid": no_bid,
        "rest_remainder": rest_remainder
    }
    return kalshi_post("/communications/quotes", body)

def get_quote(quote_id):
    """Retrieves a specific quote by ID."""
    return kalshi_get(f"/communications/quotes/{quote_id}")

def delete_quote(quote_id):
    """Deletes a quote so it can no longer be accepted."""
    return kalshi_delete(f"/communications/quotes/{quote_id}")

def accept_quote(quote_id, accepted_side):
    """Accepts a quote.
    Args:
        quote_id (str): The quote to accept.
        accepted_side (str): 'yes' or 'no'.
    """
    return kalshi_put(f"/communications/quotes/{quote_id}/accept", {"accepted_side": accepted_side})

def confirm_quote(quote_id):
    """Confirms a quote to trigger execution. Should be done after acceptance."""
    return kalshi_put(f"/communications/quotes/{quote_id}/confirm")

def get_rfqs():
    """Returns all RFQs visible to the user."""
    return kalshi_get("/communications/rfqs")

def create_rfq(market_ticker, contracts, rest_remainder=False):
    """Creates a new RFQ.
    Args:
        market_ticker (str): The market ticker you're requesting a quote for.
        contracts (int): The number of contracts to request.
        rest_remainder (bool): Whether the remainder can stay on the book.
    """
    body = {
        "market_ticker": market_ticker,
        "contracts": contracts,
        "rest_remainder": rest_remainder
    }
    return kalshi_post("/communications/rfqs", body)

def get_rfq(rfq_id):
    """Returns a specific RFQ by ID."""
    return kalshi_get(f"/communications/rfqs/{rfq_id}")

def delete_rfq(rfq_id):
    """Deletes an RFQ by ID."""
    return kalshi_delete(f"/communications/rfqs/{rfq_id}")

def get_events(limit=100, cursor=None, status=None, series_ticker=None, with_nested_markets=False):
    """
    Get a list of events with optional filtering.

    Parameters:
        limit (int): Max number of results (1-200)
        cursor (str): Used for pagination
        status (str): Filter events by status: unopened, open, closed, settled
        series_ticker (str): Filter events by series
        with_nested_markets (bool): Include nested market objects
    """
    params = {
        "limit": limit,
        "cursor": cursor,
        "status": status,
        "series_ticker": series_ticker,
        "with_nested_markets": with_nested_markets
    }
    return kalshi_get("/events", {k: v for k, v in params.items() if v is not None})

def get_event(event_ticker, with_nested_markets=False):
    """
    Get details for a specific event.

    Parameters:
        event_ticker (str): The event identifier
        with_nested_markets (bool): Include related markets
    """
    return kalshi_get(f"/events/{event_ticker}", {"with_nested_markets": with_nested_markets})

def get_markets(limit=100, cursor=None, event_ticker=None, series_ticker=None,
                max_close_ts=None, min_close_ts=None, status=None, tickers=None):
    """
    Get a list of markets with optional filters.

    Parameters:
        limit (int): Max number of results (1-1000)
        cursor (str): Used for pagination
        event_ticker (str): Filter markets belonging to this event
        series_ticker (str): Filter markets belonging to this series
        max_close_ts (int): Markets closing before this Unix timestamp
        min_close_ts (int): Markets closing after this Unix timestamp
        status (str): Filter by market status: unopened, open, closed, settled
        tickers (str): Comma-separated tickers to filter
    """
    params = {
        "limit": limit,
        "cursor": cursor,
        "event_ticker": event_ticker,
        "series_ticker": series_ticker,
        "max_close_ts": max_close_ts,
        "min_close_ts": min_close_ts,
        "status": status,
        "tickers": tickers
    }
    return kalshi_get("/markets", {k: v for k, v in params.items() if v is not None})

def get_market(ticker):
    """
    Get market details by market ticker.

    Parameters:
        ticker (str): Market ticker
    """
    return kalshi_get(f"/markets/{ticker}")

def get_market_orderbook(ticker, depth=None):
    """
    Retrieve the order book for a given market.

    Parameters:
        ticker (str): Market ticker
        depth (int): Max number of levels per side (optional)
    """
    params = {"depth": depth} if depth else None
    return kalshi_get(f"/markets/{ticker}/orderbook", params)

def get_trades(ticker=None, limit=100, cursor=None, min_ts=None, max_ts=None):
    """
    Get trade history for one or more markets.

    Parameters:
        ticker (str): Specific market ticker
        limit (int): Max number of trades to retrieve
        cursor (str): Pagination cursor
        min_ts (int): Minimum timestamp (Unix time)
        max_ts (int): Maximum timestamp (Unix time)
    """
    params = {
        "ticker": ticker,
        "limit": limit,
        "cursor": cursor,
        "min_ts": min_ts,
        "max_ts": max_ts
    }
    return kalshi_get("/markets/trades", {k: v for k, v in params.items() if v is not None})

def get_series_list(category=None, include_product_metadata=False):
    """
    Get a list of series optionally filtered by category.

    Parameters:
        category (str): Filter series by category
        include_product_metadata (bool): Include metadata
    """
    params = {
        "category": category,
        "include_product_metadata": include_product_metadata
    }
    return kalshi_get("/series/", {k: v for k, v in params.items() if v is not None})

def get_series(series_ticker):
    """
    Get a specific series by its ticker.

    Parameters:
        series_ticker (str): The series identifier
    """
    return kalshi_get(f"/series/{series_ticker}")

def get_market_candlesticks(series_ticker, ticker, start_ts, end_ts, period_interval):
    """
    Retrieve historical candlestick data for a market.

    Parameters:
        series_ticker (str): The series the market belongs to
        ticker (str): The market ticker
        start_ts (int): Unix timestamp for beginning of range
        end_ts (int): Unix timestamp for end of range
        period_interval (int): Time per candle in minutes (1, 60, or 1440)
    """
    params = {
        "start_ts": start_ts,
        "end_ts": end_ts,
        "period_interval": period_interval
    }
    return kalshi_get(f"/series/{series_ticker}/markets/{ticker}/candlesticks", params)

# ========== Exchange API Wrappers ==========

def get_exchange_announcements():
    """
    Retrieve all exchange-wide announcements.

    Returns:
        List of announcement objects with timestamps and message details.
    """
    return kalshi_get("/exchange/announcements")

def get_exchange_schedule():
    """
    Retrieve the current exchange operating schedule.

    Returns:
        Schedule object with daily hours and trading availability.
    """
    return kalshi_get("/exchange/schedule")

def get_exchange_status():
    """
    Retrieve the current operational status of the exchange.

    Returns:
        Status object including live or maintenance info.
    """
    return kalshi_get("/exchange/status")

def get_user_data_timestamp():
    """
    Retrieve the timestamp of the last backend data update for user-specific views.
    Useful for syncing between API fetches and websocket updates.

    Returns:
        Object with keys like "orders_ts", "positions_ts", etc.
    """
    return kalshi_get("/exchange/user_data_timestamp")

# ========== Milestones API Wrappers ==========

def get_milestones(limit, minimum_start_date=None, category=None, type=None, related_event_ticker=None, cursor=None):
    """
    Retrieve a list of milestones, with optional filters.

    Parameters:
        limit (int): Number of results (1 to 500) — required.
        minimum_start_date (str): ISO format datetime to filter by start date.
        category (str): Filter milestones by category.
        type (str): Filter milestones by type.
        related_event_ticker (str): Filter by related event.
        cursor (str): Pagination cursor.

    Returns:
        List of milestone metadata.
    """
    params = {
        "limit": limit,
        "minimum_start_date": minimum_start_date,
        "category": category,
        "type": type,
        "related_event_ticker": related_event_ticker,
        "cursor": cursor
    }
    return kalshi_get("/milestones/", {k: v for k, v in params.items() if v is not None})

def get_milestone(milestone_id):
    """
    Retrieve details for a specific milestone by its ID.

    Parameters:
        milestone_id (str): Unique milestone identifier.

    Returns:
        Milestone details.
    """
    return kalshi_get(f"/milestones/{milestone_id}")

# ========== Multivariate Collections API Wrappers ==========

def get_event_collections(status=None, associated_event_ticker=None, series_ticker=None, limit=100, cursor=None):
    """
    Retrieve multivariate event collections.

    Parameters:
        status (str): Filter collections by status (unopened, open, closed).
        associated_event_ticker (str): Filter by related event ticker.
        series_ticker (str): Filter by series ticker.
        limit (int): Number of results (1 to 200).
        cursor (str): Pagination cursor.

    Returns:
        List of event collection metadata.
    """
    params = {
        "status": status,
        "associated_event_ticker": associated_event_ticker,
        "series_ticker": series_ticker,
        "limit": limit,
        "cursor": cursor
    }
    return kalshi_get("/multivariate_event_collections/", {k: v for k, v in params.items() if v is not None})

def get_event_collection(collection_ticker):
    """
    Retrieve a specific multivariate event collection.

    Parameters:
        collection_ticker (str): Collection identifier.

    Returns:
        Metadata and structure of the event collection.
    """
    return kalshi_get(f"/multivariate_event_collections/{collection_ticker}")

# ========== Portfolio API Wrappers ==========

def get_balance():
    """Retrieve the current account balance."""
    return kalshi_get("/portfolio/balance")

def get_fills(ticker=None, order_id=None, min_ts=None, max_ts=None, limit=100, cursor=None):
    """Retrieve executed trade fills."""
    params = {
        "ticker": ticker,
        "order_id": order_id,
        "min_ts": min_ts,
        "max_ts": max_ts,
        "limit": limit,
        "cursor": cursor
    }
    return kalshi_get("/portfolio/fills", {k: v for k, v in params.items() if v is not None})

def get_orders(ticker=None, event_ticker=None, min_ts=None, max_ts=None, status=None, limit=100, cursor=None):
    """Retrieve all orders."""
    params = {
        "ticker": ticker,
        "event_ticker": event_ticker,
        "min_ts": min_ts,
        "max_ts": max_ts,
        "status": status,
        "limit": limit,
        "cursor": cursor
    }
    return kalshi_get("/portfolio/orders", {k: v for k, v in params.items() if v is not None})

def get_order(order_id):
    """Retrieve a specific order by ID."""
    return kalshi_get(f"/portfolio/orders/{order_id}")

def create_order(order_data):
    """Submit a new order.

    Args:
        order_data (dict): Must include fields like ticker, side, type, count, and price (yes_price or no_price).

    Returns:
        Confirmation with order ID.
    """
    return kalshi_post("/portfolio/orders", order_data)

def batch_create_orders(order_list):
    """Submit multiple orders in a single request (advanced users only)."""
    return kalshi_post("/portfolio/orders/batched", {"orders": order_list})

def cancel_order(order_id):
    """Cancel a specific order by ID."""
    return kalshi_delete(f"/portfolio/orders/{order_id}")

def batch_cancel_orders(order_ids):
    """Cancel multiple orders by ID (advanced users only)."""
    return kalshi_delete("/portfolio/orders/batched", {"ids": order_ids})

def decrease_order(order_id, reduce_by=None, reduce_to=None):
    """Decrease the number of contracts in an existing order."""
    body = {"reduce_by": reduce_by, "reduce_to": reduce_to}
    return kalshi_post(f"/portfolio/orders/{order_id}/decrease", {k: v for k, v in body.items() if v is not None})

def get_positions(ticker=None, event_ticker=None, count_filter=None, settlement_status="unsettled", limit=100, cursor=None):
    """Get all market positions."""
    params = {
        "ticker": ticker,
        "event_ticker": event_ticker,
        "count_filter": count_filter,
        "settlement_status": settlement_status,
        "limit": limit,
        "cursor": cursor
    }
    return kalshi_get("/portfolio/positions", {k: v for k, v in params.items() if v is not None})

def get_portfolio_settlements(limit=100, min_ts=None, max_ts=None, cursor=None):
    """Retrieve historical portfolio settlements."""
    params = {
        "limit": limit,
        "min_ts": min_ts,
        "max_ts": max_ts,
        "cursor": cursor
    }
    return kalshi_get("/portfolio/settlements", {k: v for k, v in params.items() if v is not None})

def get_total_resting_order_value():
    """Retrieve the total value of resting orders (FCM members only)."""
    return kalshi_get("/portfolio/summary/total_resting_order_value")

# ========== Structured Targets API Wrappers ==========

def get_structured_target(structured_target_id):
    """Retrieve a structured target by ID.

    Args:
        structured_target_id (str): The ID of the structured target.

    Returns:
        JSON response containing structured target metadata.
    """
    return kalshi_get(f"/structured_targets/{structured_target_id}")

In [14]:
get_market_orderbook(ticker='KXNEXTPOPE-35-MGRE', depth=10)

{'orderbook': {'yes': [[1, 25200]],
  'no': [[85, 1228],
   [86, 1176],
   [87, 967],
   [88, 941],
   [89, 680],
   [90, 3950],
   [95, 3000],
   [96, 5333],
   [97, 22800],
   [98, 22377]]}}

In [3]:
get_markets(limit=10, cursor=None, event_ticker=None, series_ticker=None,
                max_close_ts=None, min_close_ts=None, status=None, tickers=None)

{'markets': [{'ticker': 'KXCPI-25MAY-T0.4',
   'event_ticker': 'KXCPI-25MAY',
   'market_type': 'binary',
   'title': 'Will CPI rise more than 0.4% in May 2025?',
   'subtitle': '0.4%',
   'yes_sub_title': 'Above 0.4%',
   'no_sub_title': 'Above 0.4%',
   'open_time': '2025-04-24T16:00:59Z',
   'close_time': '2025-06-11T12:25:00Z',
   'expected_expiration_time': '2025-06-11T14:00:00Z',
   'expiration_time': '2025-06-18T14:00:00Z',
   'latest_expiration_time': '2025-06-18T14:00:00Z',
   'settlement_timer_seconds': 3600,
   'status': 'active',
   'response_price_units': 'usd_cent',
   'notional_value': 100,
   'tick_size': 1,
   'yes_bid': 13,
   'yes_ask': 17,
   'no_bid': 83,
   'no_ask': 87,
   'last_price': 0,
   'previous_yes_bid': 0,
   'previous_yes_ask': 0,
   'previous_price': 0,
   'volume': 0,
   'volume_24h': 0,
   'liquidity': 101480,
   'open_interest': 0,
   'result': '',
   'can_close_early': True,
   'expiration_value': '',
   'category': '',
   'risk_limit_cents': 0,
  

In [4]:
get_series(series_ticker="KXRATECUTS")

{'series': {'ticker': 'KXRATECUTS',
  'frequency': 'annual',
  'title': 'Number of rate cuts',
  'category': 'Economics',
  'tags': ['Interest rates'],
  'settlement_sources': [{'url': 'https://www.federalreserve.gov/monetarypolicy/openmarket.htm',
    'name': 'Federal Reserve'}],
  'contract_url': 'https://kalshi-public-docs.s3.us-east-1.amazonaws.com/regulatory/product-certifications/RATECUTS.pdf'}}